# Feature Engineering - Checkins (Versão Spark)

**Conversão do notebook pandas → Spark**

Este notebook:
1. Lê checkins da camada Bronze (Spark)
2. Processa e agrega por business_id
3. Faz join com Business e Tips da Silver
4. Aplica normalização (log + MinMaxScaler)
5. Salva features finais na Gold layer

**Vantagens vs Pandas:**
- ✅ Sem problemas de memória
- ✅ Processamento paralelo
- ✅ Acessa datalake diretamente
- ✅ Escalável

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, split, explode, size, log1p, 
    when, coalesce, lit, broadcast
)
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

In [2]:
# ⚡ CONFIGURAÇÃO SPARK OTIMIZADA
spark = SparkSession.builder \
    .appName("Checkin Feature Engineering") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print(f"✅ Spark version: {spark.version}")

✅ Spark version: 3.5.0


In [3]:
# Configuração de Paths
BASE_PATH = '/home/jovyan/work'
DATA_PATH = f'{BASE_PATH}/data'
BRONZE_PATH = f'{DATA_PATH}/bronze'
SILVER_PATH = f'{DATA_PATH}/silver'
GOLD_PATH = f'{DATA_PATH}/gold'

print(f"🥉 Bronze: {BRONZE_PATH}")
print(f"🥈 Silver: {SILVER_PATH}")
print(f"🥇 Gold: {GOLD_PATH}")

🥉 Bronze: /home/jovyan/work/data/bronze
🥈 Silver: /home/jovyan/work/data/silver
🥇 Gold: /home/jovyan/work/data/gold


## 1. Carregar Dados do Datalake

In [4]:
print("📥 [1/5] Carregando dados do datalake...\n")

# 1. Checkin (Bronze)
print("   📦 Carregando checkins...")
df_checkin = spark.read.parquet(f"{BRONZE_PATH}/checkin")
print(f"      ✅ Checkins: {df_checkin.count():,} registros")

# 2. Business (Silver) - Já filtrado
print("   📦 Carregando business...")
df_business = spark.read.parquet(f"{SILVER_PATH}/business")
print(f"      ✅ Business: {df_business.count():,} registros")

print("   📦 Carregando tips...")
try:
    df_tips = spark.read.parquet(f"{SILVER_PATH}/tip_features_business")
    print(f"      ✅ Tips: {df_tips.count():,} registros")
    has_tips = True
except:
    print("      ⚠️ Tips não encontrado, criando features vazias")
    has_tips = False

print("\n✅ Dados carregados!")

📥 [1/5] Carregando dados do datalake...

   📦 Carregando checkins...
      ✅ Checkins: 131,930 registros
   📦 Carregando business...
      ✅ Business: 64,645 registros
   📦 Carregando tips...
      ✅ Tips: 44,904 registros

✅ Dados carregados!


In [5]:
# Verificar schema dos checkins
print("📋 Schema do Checkin:\n")
df_checkin.printSchema()

print("\n📊 Amostra:")
df_checkin.show(5, truncate=50)

📋 Schema do Checkin:

root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)


📊 Amostra:
+----------------------+--------------------------------------------------+
|           business_id|                                              date|
+----------------------+--------------------------------------------------+
|9D0dAIUpSoC91NKHK5OAPQ|2015-02-13 22:42:58, 2015-04-24 20:45:29, 2015-...|
|9D1Vuxuh-hvwkYBrqje1nA|2010-07-25 22:42:21, 2010-08-22 16:26:12, 2010-...|
|9D3mvHmxQouaOcizFKg1dg|2011-07-24 01:09:08, 2011-10-04 15:41:08, 2012-...|
|9D4gd_k6s8xzeWKXrqYcmA|2012-01-30 00:55:35, 2013-02-02 13:58:34, 2013-...|
|9D6YY6_J1diGiBOQJJ7juA|2014-02-24 18:49:29, 2015-04-28 20:01:29, 2021-...|
+----------------------+--------------------------------------------------+
only showing top 5 rows



## 2. Processar Checkins

O campo `date` é uma string com múltiplas datas separadas por vírgula.  
Precisamos contar quantas datas existem para cada business.

In [6]:
print("\n🔧 [2/5] Processando checkins...\n")

# Estratégia: Contar quantas vírgulas tem na string + 1
# Ex: "2020-01-01, 2020-01-02" → 2 checkins

# Filtrar apenas business_ids válidos (que estão na Silver)
valid_business_ids = df_business.select('business_id')

df_checkin_filtered = df_checkin.join(
    broadcast(valid_business_ids),
    on='business_id',
    how='inner'
)

print(f"   ✅ Filtrados para business válidos: {df_checkin_filtered.count():,}")

# Contar checkins: split por vírgula e pegar tamanho
df_checkin_count = df_checkin_filtered \
    .withColumn('checkin_total', size(split(col('date'), ',')))

# Agregar por business_id
df_checkin_agg = df_checkin_count \
    .groupBy('business_id') \
    .agg(
        F.sum('checkin_total').alias('checkin_total')
    )

print(f"   ✅ Checkins agregados: {df_checkin_agg.count():,} businesses")

# Estatísticas
print("\n   📊 Estatísticas de checkins:")
df_checkin_agg.select('checkin_total').summary().show()


🔧 [2/5] Processando checkins...

   ✅ Filtrados para business válidos: 56,107
   ✅ Checkins agregados: 56,107 businesses

   📊 Estatísticas de checkins:
+-------+------------------+
|summary|     checkin_total|
+-------+------------------+
|  count|             56107|
|   mean|139.09768834548274|
| stddev| 579.6346304425077|
|    min|                 1|
|    25%|                 6|
|    50%|                22|
|    75%|                89|
|    max|             52144|
+-------+------------------+



## 3. Join com Business e Tips

In [7]:
print("\n🔗 [3/5] Unificando tabelas...\n")

# A. Começar com Business (base)
df_final = df_business

# B. Left Join com Checkins
df_final = df_final.join(
    df_checkin_agg,
    on='business_id',
    how='left'
)

# Preencher nulos com 0 (businesses sem checkin)
df_final = df_final.fillna({'checkin_total': 0})

print(f"   ✅ Checkins integrados")

# C. Left Join com Tips (se existir)
if has_tips:
    df_final = df_final.join(
        df_tips,
        on='business_id',
        how='left'
    )
    
    # Preencher nulos
    df_final = df_final.fillna({
        'tip_count_log': 0,
        'recency_score': 0
    })
    
    print(f"   ✅ Tips integrados")
else:
    # Criar colunas de tips com zeros
    df_final = df_final \
        .withColumn('tip_count_log', lit(0.0)) \
        .withColumn('recency_score', lit(0.0))
    
    print(f"   ✅ Colunas de tips criadas (zeros)")

print(f"\n   📊 Total de registros: {df_final.count():,}")


🔗 [3/5] Unificando tabelas...

   ✅ Checkins integrados
   ✅ Tips integrados

   📊 Total de registros: 64,645


In [ ]:
df_final

## 4. Feature Engineering - Normalização

In [8]:
print("\n🧪 [4/5] Aplicando transformações...\n")

# A. Log transformation no checkin_total
print("   📐 Aplicando log1p no checkin_total...")
df_final = df_final.withColumn(
    'checkin_total_log',
    log1p(col('checkin_total'))
)

# B. Normalizar review_count (se existir)
if 'review_count' in df_final.columns:
    print("   📐 Aplicando log1p no review_count...")
    df_final = df_final.withColumn(
        'review_count_log',
        log1p(col('review_count'))
    )

# C. Normalizar stars (se existir) - de 1-5 para 0-1
if 'stars' in df_final.columns:
    print("   📐 Normalizando stars (1-5 → 0-1)...")
    df_final = df_final.withColumn(
        'stars',
        (col('stars') - 1) / 4  # Min=1, Max=5 → (x-1)/4
    )

print("\n   ✅ Transformações aplicadas!")


🧪 [4/5] Aplicando transformações...

   📐 Aplicando log1p no checkin_total...
   📐 Normalizando stars (1-5 → 0-1)...

   ✅ Transformações aplicadas!


In [9]:
print("\n📊 Aplicando MinMaxScaler...\n")

# Colunas para normalizar
cols_to_normalize = ['checkin_total_log']

if 'review_count_log' in df_final.columns:
    cols_to_normalize.append('review_count_log')

# Aplicar MinMaxScaler
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

for col_name in cols_to_normalize:
    print(f"   ⚙️  Normalizando {col_name}...")
    
    # 1. Criar Vector (MinMaxScaler precisa de Vector)
    assembler = VectorAssembler(
        inputCols=[col_name],
        outputCol=f"{col_name}_vec"
    )
    df_final = assembler.transform(df_final)
    
    # 2. Aplicar MinMaxScaler
    scaler = MinMaxScaler(
        inputCol=f"{col_name}_vec",
        outputCol=f"{col_name}_scaled"
    )
    scaler_model = scaler.fit(df_final)
    df_final = scaler_model.transform(df_final)
    
    # 3. Extrair valor do Vector de volta para coluna simples
    from pyspark.sql.functions import udf
    from pyspark.ml.linalg import VectorUDT, DenseVector
    
    def extract_value(vector):
        if vector is None:
            return 0.0
        return float(vector[0])
    
    extract_udf = udf(extract_value, DoubleType())
    
    df_final = df_final.withColumn(
        col_name,
        extract_udf(col(f"{col_name}_scaled"))
    )
    
    # 4. Dropar colunas temporárias
    df_final = df_final.drop(f"{col_name}_vec", f"{col_name}_scaled")

print("\n   ✅ MinMaxScaler aplicado!")


📊 Aplicando MinMaxScaler...

   ⚙️  Normalizando checkin_total_log...

   ✅ MinMaxScaler aplicado!


In [17]:
df_final.show(1, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------
 business_id       | qWH526XL2vBmrOKCNcVthw                                                                                                         
 name              | Green Hills Pharmacy                                                                                                           
 city              | Nashville                                                                                                                      
 categories        | Vocational & Technical School, Specialty Schools, Adult Education, Education, Drugstores, Health & Medical, Pharmacy, Shopping 
 stars             | 0.875                                                                                                                          
 is_open           | 1                                                                                    

In [ ]:
# Mostrar resultado
print("\n📊 Features finais:\n")

cols_to_show = ['business_id', 'name', 'city', 'stars', 
                'checkin_total', 'checkin_total_log', 
                'tip_count_log', 'recency_score']

# Adicionar review_count_log se existir
if 'review_count_log' in df_final.columns:
    cols_to_show.append('review_count_log')

# Filtrar apenas colunas que existem
cols_to_show = [c for c in cols_to_show if c in df_final.columns]

df_final.select(cols_to_show).show(10, truncate=30)

# Estatísticas das features normalizadas
print("\n📈 Estatísticas das features normalizadas:")
df_final.select('checkin_total_log', 'tip_count_log', 'recency_score').summary().show()

## 5. Salvar na Gold Layer

In [18]:
print("\n💾 [5/5] Salvando na Silver Layer...\n")

import os
import shutil

# Criar diretório Silver se não existir
os.makedirs(SILVER_PATH, exist_ok=True)

output_path = f'{SILVER_PATH}/item_features_enriched'

# Remover se existir
if os.path.exists(output_path):
    shutil.rmtree(output_path)
    print(f"   🗑️  Removido arquivo antigo: {output_path}")

# Salvar
df_final \
    .repartition(10) \
    .write \
    .mode('overwrite') \
    .option('compression', 'snappy') \
    .parquet(output_path)

print(f"\n{'='*60}")
print(f"✅ ARQUIVO FINAL GERADO: {output_path}")
print(f"   📊 Dimensões: {df_final.count():,} linhas x {len(df_final.columns)} colunas")
print(f"   📦 Partições: 10")
print(f"   🗜️  Compressão: SNAPPY")
print(f"{'='*60}")


💾 [5/5] Salvando na Silver Layer...


✅ ARQUIVO FINAL GERADO: /home/jovyan/work/data/silver/item_features_enriched
   📊 Dimensões: 64,645 linhas x 11 colunas
   📦 Partições: 10
   🗜️  Compressão: SNAPPY


In [20]:
# Verificar arquivo salvo
print("\n🔍 Verificando arquivo salvo...\n")

df_verify = spark.read.parquet(output_path)

print(f"✅ Arquivo lido com sucesso!")
print(f"   Total de registros: {df_verify.count():,}")
print(f"\n📋 Colunas:")
for col in df_verify.columns:
    print(f"   - {col}")

print("\n📊 Amostra dos dados:")
df_verify.show(5, truncate=10)


🔍 Verificando arquivo salvo...

✅ Arquivo lido com sucesso!
   Total de registros: 64,645

📋 Colunas:
   - business_id
   - name
   - city
   - categories
   - stars
   - is_open
   - log_review_count
   - checkin_total
   - tip_count_log
   - recency_score
   - checkin_total_log

📊 Amostra dos dados:
+-----------+----------+----------+----------+-----+-------+----------------+-------------+-------------+-------------+-----------------+
|business_id|      name|      city|categories|stars|is_open|log_review_count|checkin_total|tip_count_log|recency_score|checkin_total_log|
+-----------+----------+----------+----------+-----+-------+----------------+-------------+-------------+-------------+-----------------+
| 0OVkfxm...|AMC DIN...|Philade...|Cinema,...|0.625|      1|      3.68887...|           43|   0.09681...|   0.54508...|       0.34839...|
| 7RYxPJo...|Peach P...|Philade...|Event P...|  1.0|      1|      2.19722...|            0|          0.0|          0.0|              0.0|
| Y-oR

In [ ]:
# Limpeza
print("\n🧹 Limpando cache...")
spark.catalog.clearCache()
print("✅ Cache limpo!")

print("\n🎉 PROCESSAMENTO COMPLETO!")

---

## 📊 Resumo do Pipeline

**Input:**
- `bronze/checkin` - Checkins brutos
- `silver/business` - Business filtrados
- `silver/tip_features_business` - Features de tips (opcional)

**Transformações:**
1. ✅ Contagem de checkins por business
2. ✅ Join com business e tips
3. ✅ Log transformation (checkin_total, review_count)
4. ✅ Normalização stars (1-5 → 0-1)
5. ✅ MinMaxScaler em features numéricas

**Output:**
- `gold/item_features_enriched` - Features prontas para modelo

**Features Finais:**
- `business_id` - ID único
- `name` - Nome do estabelecimento
- `city` - Cidade
- `categories` - Categorias
- `stars` - Rating normalizado [0-1]
- `checkin_total` - Total de checkins (raw)
- `checkin_total_log` - Checkins normalizados [0-1]
- `review_count_log` - Reviews normalizados [0-1]
- `tip_count_log` - Tips normalizados [0-1]
- `recency_score` - Score de recência [0-1]

---